In [104]:
import pandas as pd
import numpy as np

In [105]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score, roc_auc_score

In [106]:
dats_source_path = '../data_set/Hr/'

train_x = pd.read_csv(dats_source_path+"HRdata_X_train.csv")
train_y = pd.read_csv(dats_source_path+"HRdata_y_train.csv")
test_x = pd.read_csv(dats_source_path+"HRdata_X_test.csv")

display(train_x)
display(train_y)
#display(test_x)

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,25298,city_138,0.836,Male,No relevent experience,Full time course,High School,NaN,5,100-500,Pvt Ltd,1,45
1,4241,city_160,0.920,Male,No relevent experience,Full time course,High School,NaN,5,NaN,NaN,1,17
2,24086,city_57,0.866,Male,No relevent experience,no_enrollment,Graduate,STEM,10,NaN,NaN,1,50
3,26773,city_16,0.910,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,>4,135
4,32325,city_143,0.740,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12447,29597,city_21,0.624,Male,Has relevent experience,no_enrollment,Graduate,STEM,<1,50-99,Pvt Ltd,1,55
12448,27821,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,10000+,Pvt Ltd,>4,7
12449,21812,city_160,0.920,Female,Has relevent experience,no_enrollment,Graduate,STEM,6,50-99,Pvt Ltd,1,36
12450,28416,city_160,0.920,Male,Has relevent experience,Full time course,Graduate,STEM,10,NaN,NaN,2,41


,enrollee_id,target
0,25298,0.0
1,4241,1.0
2,24086,0.0
3,26773,0.0
4,32325,1.0
...,...,...
12447,29597,1.0
12448,27821,0.0
12449,21812,0.0
12450,28416,0.0


In [107]:
#train set 으로 사용할 df 중 예측에 사용되지 않는 컬럼
drop_col = ['enrollee_id','city']

#Test set의 결과 파일에 출력할 CustomerId 저장
test_id = test_x['enrollee_id']

#Y의 Train set
y = train_y['target']

In [108]:
test_x.isna().sum()

enrollee_id                  0
city                         0
city_development_index       0
gender                    1591
relevent_experience          0
enrolled_university        129
education_level            145
major_discipline           947
experience                  28
company_size              2086
company_type              2159
last_new_job               150
training_hours               0
dtype: int64

In [109]:
train_x = train_x.fillna(method='pad')
test_x = test_x.fillna(method='pad')

train_x = train_x.fillna(method='bfill')
test_x = test_x.fillna(method='bfill')


In [110]:
display(test_x.isna().sum())
display(train_x.isna().sum())

enrollee_id               0
city                      0
city_development_index    0
gender                    0
relevent_experience       0
enrolled_university       0
education_level           0
major_discipline          0
experience                0
company_size              0
company_type              0
last_new_job              0
training_hours            0
dtype: int64

enrollee_id               0
city                      0
city_development_index    0
gender                    0
relevent_experience       0
enrolled_university       0
education_level           0
major_discipline          0
experience                0
company_size              0
company_type              0
last_new_job              0
training_hours            0
dtype: int64

In [111]:
train_x_drop = train_x.drop(columns=drop_col)
test_x_drop = test_x.drop(columns=drop_col)

train_x_dum = pd.get_dummies(train_x_drop)
test_x_dum = pd.get_dummies(test_x_drop)

test_x_dum = test_x_dum[train_x_dum.columns]

In [112]:
from sklearn.model_selection import train_test_split

X_train, X_valid, Y_train, Y_valid = train_test_split(train_x_dum, y, test_size = 0.3, random_state = 38, stratify = y)

In [113]:
rfc = RandomForestClassifier(random_state=78)
rfc.fit(X_train, Y_train)

valid_pred = rfc.predict(X_valid)
valid_pred_proba = rfc.predict_proba(X_valid)[:,1]

print('valid f1_score: ', f1_score(Y_valid, valid_pred))
print('valid roc_auc_score: ', roc_auc_score(Y_valid, valid_pred_proba))

valid f1_score:  0.4120734908136483
valid roc_auc_score:  0.7285765506664299


In [114]:
dtc = DecisionTreeClassifier(random_state=78)
dtc.fit(X_train, Y_train)

valid_pred = dtc.predict(X_valid)
valid_pred_proba = dtc.predict_proba(X_valid)[:,1]

print('valid f1_score: ', f1_score(Y_valid, valid_pred))
print('valid roc_auc_score: ', roc_auc_score(Y_valid, valid_pred_proba))

valid f1_score:  0.3954214360041623
valid roc_auc_score:  0.5946670299327141


In [115]:
gbc = GradientBoostingClassifier(random_state=78)
gbc.fit(X_train, Y_train)

valid_pred = gbc.predict(X_valid)
valid_pred_proba = gbc.predict_proba(X_valid)[:,1]

print('valid f1_score: ', f1_score(Y_valid, valid_pred))
print('valid roc_auc_score: ', roc_auc_score(Y_valid, valid_pred_proba))

valid f1_score:  0.4902821316614421
valid roc_auc_score:  0.7607770628103323


In [116]:
test_pred = gbc.predict(test_x_dum)

result = pd.DataFrame({"CustomerId": test_id,
              "Target": test_pred})



In [117]:
result

,CustomerId,Target
0,7129,0.0
1,31037,0.0
2,22179,0.0
3,29724,0.0
4,17977,1.0
...,...,...
6701,3601,0.0
6702,2745,0.0
6703,18520,0.0
6704,10067,0.0
